In [1]:
from neo4j import GraphDatabase

In [2]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [3]:
# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "12345678")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

conn = Neo4jConnection(uri=URI, user=AUTH[0], pwd=AUTH[1])

<h2>Load Data</h2>

In [4]:
query = '''LOAD CSV WITH HEADERS FROM 'file:///sampleArticle.csv' AS line FIELDTERMINATOR ','
           MERGE (p:Paper {id:line.articleID, title: line.title, year: toInteger(line.year), corresponding_author: line.correspondingAuthor, abstract: line.abstract})
           MERGE (v:Volume {id: line.volume_id, number: line.volume})
           MERGE (p)-[:BELONGS_TO]->(v)
           
           MERGE (j:Journal {id: line.issn, name: line.journal})
           MERGE (v)-[:PUBLISHED_IN {year: toInteger(line.year)}]->(j)
           
           FOREACH (n IN CASE WHEN line.edition_id IS NOT NULL THEN [1] ELSE [] END |
            MERGE (e:Edition {id: line.edition_id, number: line.edition_number})
            MERGE (p)-[:PRESENTED_IN {year: line.edition_year}]->(e)

            MERGE (c:Conference {id: line.conference})
            MERGE (e)-[:OF]->(c)

            MERGE (ct:City {name: line.city}) 
            MERGE (e)-[:HOSTED_IN {from: line.start_date, to: line.end_date}]->(ct))

           WITH p, line
           UNWIND split(line.authors, '|') as name
           MERGE (a:Author {name: name})
           MERGE (a)-[:WROTE]->(p)

           WITH p, line
           UNWIND split(line.reviewers, '|') as reviewer
           MATCH (rev:Author)
           WHERE rev.name = reviewer
           WITH rev, p, line
           MERGE (rev)-[:REVIEWS]->(p)

           WITH p, line
           UNWIND split(line.keywords, '|') as word
           MERGE (key:Keyword {word: word})
           MERGE (p)-[:RELATED_TO]->(key)

           WITH p, line
           UNWIND split(line.citations, '|') as cite
           MATCH (citedPaper:Paper), (paper:Paper)
           WHERE citedPaper.id = cite and paper.id = line.articleID
           WITH citedPaper, paper 
           MERGE (paper)-[:CITES]->(citedPaper)

           '''

conn.query(query=query)

[]

In [5]:
conn.close()